# Examples of Use for rings.py Module

This module is the crown jewel of ZSE. I have developed algorithms using graph theory that are able to find rings in zeolite frameworks. There are a number of uses for this information. I use ring finding to in the zse.cation.py module to place cations in each of the rings associated with a T-site. Our group uses the rings associated with an oxygen or a T-site to classify them in a more descriptive way than just saying O1 or T4. Most recently I have been using the rings associated with an oxygen site, tetrahedral site, and entire framework as descriptors for neural networks. I have been able to predict deprotonoation energies, and proton location energies using these methods.

## rings.get_orings( )
This is the basic building block of all the ring finding in ZSE. We can use this function to find all the rings associated with an oxygen site in a zeolite framework. \
**Note** This works best if you remove any adsorbates from your framework first. 
### Inputs
**z** is an atoms object containing your zeolite framework \
**index** is the index of the oxygen atom you want to find the rings for \
**code** is the IZA framework code for the type of zeolite you are working with \
### Outputs
**ring_list** is a list of all the ring sizes associated with **index** \
**paths** contains the indices of the atoms that make up each ring \
**ring_atoms** is an atoms object that only contains atoms in the rings

In [1]:
from zse.collections import *
from zse.rings import *
from zse.utilities import *
from ase.visualize import view

In [3]:
z = framework('CHA')
ring_list, paths, ring_atoms = get_orings(z,0,'CHA')

In [4]:
print('Ring Size \t Path')
for r,p in zip(ring_list,paths):
    print('{0} \t\t {1}'.format(r,p))

Ring Size 	 Path
6 		 [0, 81, 255, 291, 219, 303, 690, 510, 438, 516, 468, 72]
4 		 [0, 81, 63, 420, 159, 645, 486, 72]
4 		 [0, 81, 27, 91, 10, 100, 18, 72]


## Notes
A couple things to note here is that when people discuss rings in zeolites, a 6-membered ring (6-MR) contains 6 tetrahedral sites. So a 6-MR actually contains 12 atoms total. \
This particular oxygen (index = 0) is associated with three rings: a 6-MR and two 4-MRs. \
We can also visualize these rings below.

In [6]:
view(ring_atoms)

<img src="figures/cha_o_rings.png" style="width: 400px;"/>

## rings.get_trings( )
So now that you are all experts at understanding the rings in zeolites we can move on to finding the rings associated with a tetrahedral site. To do this we will simply find all the rings associated with each oxygen bound to that tsite. 

Inputs and outputs are idential to get_orings( ) above.

In [8]:
ring_list,paths,ring_atoms = get_trings(z,101,'CHA')

In [9]:
print('Ring Size \t Path')
for r,p in zip(ring_list,paths):
    print('{0} \t\t {1}'.format(r,p))

Ring Size 	 Path
8 		 [19, 101, 62, 80, 26, 102, 45, 522, 461, 515, 497, 530, 463, 85, 37, 73]
8 		 [19, 101, 50, 95, 34, 304, 286, 307, 243, 297, 39, 75, 21, 106, 55, 73]
6 		 [11, 101, 50, 95, 14, 107, 53, 98, 17, 104, 47, 92]
4 		 [50, 101, 62, 80, 8, 86, 68, 95]
4 		 [11, 101, 62, 80, 44, 89, 71, 92]
4 		 [11, 101, 19, 73, 1, 82, 28, 92]


## Notes
Here we can see that this T-site is associated with 6 rings: two 8-MRs, one 6-MR, and three 4-MRs. Since chabazite has only one unique T-site, every T-site in this framework will have the same ring classification. 

I've included a screenshot of the ring_atoms below. It's difficult to see all the rings in a static image, so I encourage you to do this for yourself and play with interactive viewer. 

In [10]:
view(ring_atoms)

<img src="figures/cha_t_rings.png" style="width: 400px;"/>

## rings.get_fwrings( )
The only thing left now is to find all the unique rings in a framework. To do this we will find all the rings associated with each type of unique oxygen in the framwork. Add all the rings together and use some tricks to remove duplicates. 

The inputs and outputs of this function are slightly different from what we had above. 

### Inputs
**code** IZA framework code for the zeolite you are using
### Outputs
Note: \
All outputs are in dictionary form where the key is the size of ring. This will make more sense in the example.

**index_paths** Contains all the indices for the atoms in each ring \
**label_paths** Contains all the site labels for the atoms in each ring \
**trajectories** Contains an atoms object of each ring type for visualization \ 


In [19]:
# I'm going to use a different framework from CHA here
# Since CHA only has one unique T-site, it doesn't have many unique rings

index_paths, label_paths, trajectories = get_fwrings('AEI')

In [21]:
ring_sizes = get_ring_sizes('AEI')
for r in ring_sizes:
    print('Unique {0}-MRs: {1}'.format(r,len(label_paths[r])))
    for p in label_paths[r]:
        print(p)

Unique 8-MRs: 2
['O2', 'T1', 'O4', 'T3', 'O7', 'T2', 'O3', 'T1', 'O2', 'T1', 'O3', 'T2', 'O7', 'T3', 'O4', 'T1']
['O6', 'T2', 'O7', 'T3', 'O8', 'T3', 'O7', 'T2', 'O6', 'T2', 'O7', 'T3', 'O8', 'T3', 'O7', 'T2']
Unique 6-MRs: 1
['O1', 'T1', 'O4', 'T3', 'O5', 'T2', 'O6', 'T2', 'O5', 'T3', 'O4', 'T1']
Unique 4-MRs: 4
['O1', 'T1', 'O3', 'T2', 'O6', 'T2', 'O3', 'T1']
['O1', 'T1', 'O2', 'T1', 'O1', 'T1', 'O2', 'T1']
['O3', 'T2', 'O5', 'T3', 'O8', 'T3', 'O4', 'T1']
['O5', 'T2', 'O7', 'T3', 'O5', 'T2', 'O7', 'T3']


## Notes
So here we can see that the AEI framework has two unique 8-MRs, one unique 6-MR, and four unique 4-MRs. You can look at the ordering of the T-sites and O-sites in the rings to verrify that they are in fact unique.

## Using the Trajectories
To use the trajectories, you must first call the size rings you are interested in: i.e. trajectories[8] will contain both the unique 10-MRs. 

In [22]:
view(trajectories[8])

<img src="figures/aei_8-2.png" width="300" /> <img src="figures/aei_8-1.png" width="300" />


## rings.get_vertex_symbols( )

This is a method to find the shortest paths between each of the 6 possible oxygen-oxygen pairs around a T-site. The vertex symbol for each framework in the IZA is already known, but this toll will save the atoms objects for each oxygen-oxygen pair so the rings can be visualized. 

### Inputs
**code** IZA framework code for the zeolite you are using \
**index** Index of the T-site you would like to characterize
### Outputs
**vertex_symbols** Dictionary: keys are the oxy-oxy vertex symbol, values the indices of the atoms that make up the path \
**traj** Collection of atoms objects. There should be 6, one for each oxy-oxy pair, showing the rings connecting those two oxygen. \ 



In [2]:
# pick a framework
code = 'DFT'

# get the indices of the T-sites for a standard IZA unit cell
ts,tm,inds = get_tsites(code)
index = inds[0]

# run the function
vertex_symbols, traj = get_vertex_symbols(code,index)

In [4]:
# Let's check out the results:
for v in vertex_symbols:
    print(v)
    for path in vertex_symbols[v]:
        print(int(len(path)/2),path)

1:O1-O1
4 [0, 21, 4, 18, 2, 23, 6, 16]
2:O1-O2
6 [0, 21, 4, 18, 10, 68, 51, 67, 55, 70, 8, 16]
6 [0, 21, 4, 18, 10, 68, 53, 65, 49, 70, 8, 16]
3:O1-O3
8 [0, 21, 11, 19, 7, 22, 302, 308, 291, 307, 299, 309, 292, 306, 300, 16]
8 [0, 21, 11, 19, 7, 22, 302, 308, 293, 305, 297, 311, 290, 306, 300, 16]
8 [0, 21, 15, 239, 222, 232, 516, 522, 506, 527, 303, 309, 292, 306, 300, 16]
4:O1-O2
6 [6, 23, 2, 18, 10, 68, 51, 67, 55, 70, 8, 16]
6 [6, 23, 2, 18, 10, 68, 53, 65, 49, 70, 8, 16]
5:O1-O3
8 [6, 23, 9, 17, 1, 22, 302, 308, 291, 307, 299, 309, 292, 306, 300, 16]
8 [6, 23, 9, 17, 1, 22, 302, 308, 293, 305, 297, 311, 290, 306, 300, 16]
8 [6, 23, 87, 93, 72, 88, 372, 378, 364, 381, 375, 311, 290, 306, 300, 16]
6:O2-O3
4 [8, 70, 350, 356, 298, 306, 300, 16]


### Analyze Results
So the single T-site in DFT has a vertex symbol of: $4\cdot6_2\cdot8_3\cdot6_2\cdot8_3\cdot4$ \
This is in agreement with what is shown on the [IZA Structure Database](https://america.iza-structure.org/IZA-SC/framework_cs.php?STC=DFT)